Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/MachineLearningNotebooks/how-to-use-azureml/explain-model/azure-integration/remote-explanation/explain-model-on-amlcompute.png)

In [ ]:
# Check core SDK version number
import azureml.core

print("SDK version:", azureml.core.VERSION)

## Initialize workspace

Load and initialize a [Workspace](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#workspace) object from the existing workspace you created in the prerequisites step. `Workspace.from_config()` creates a workspace object from the details stored in `config.json`

In [ ]:
import os

subscription_id = os.getenv("SUBSCRIPTION_ID", default="<subsciption-id>")
resource_group = os.getenv("RESOURCE_GROUP", default="<resource-group>")
workspace_name = os.getenv("WORKSPACE_NAME", default="<workspace-name>")
workspace_region = os.getenv("WORKSPACE_REGION", default="<workspace_region>")

In [ ]:
from azureml.core import Workspace

try:
    ws = Workspace(subscription_id = subscription_id, resource_group = resource_group, workspace_name = workspace_name)
    # write the details of the workspace to a configuration file to the notebook library
    ws.write_config()
    print("Workspace configuration succeeded.")
except:
    print("Workspace not accessible. Creting new workspace...")
    from azureml.core import Workspace

    # Create the workspace using the specified parameters
    ws = Workspace.create(name = workspace_name,
                          subscription_id = subscription_id,
                          resource_group = resource_group, 
                          location = workspace_region,
                          create_resource_group = True,
                          exist_ok = True)
    ws.get_details()

    # write the details of the workspace to a configuration file to the notebook library
    ws.write_config()

In [ ]:
from azureml.core import Workspace

ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep='\n')

datastore = ws.get_default_datastore()
print("Default datastore's name: {}".format(datastore.name))

## Create An Experiment

**Experiment** is a logical container in an Azure ML Workspace. It hosts run records which can include run metrics and output artifacts from your experiments.

In [ ]:
from azureml.core import Experiment
experiment_name = 'gpu-shap-on-amlcompute'
experiment = Experiment(workspace=ws, name=experiment_name)

### Provision a compute target

You can provision an AmlCompute resource by simply defining two parameters thanks to smart defaults. By default it autoscales from 0 nodes and provisions dedicated VMs to run your job in a container. This is useful when you want to continously re-use the same target, debug it between jobs or simply share the resource with other users of your workspace.

* `vm_size`: VM family of the nodes provisioned by AmlCompute. RAPIDS requires NVIDIA Pascal or newer architecture, you will need to specify compute targets from one of `NC_v2`, `NC_v3`, `ND` or `ND_v2` [GPU virtual machines in Azure](https://docs.microsoft.com/en-us/azure/virtual-machines/sizes-gpu); these are VMs that are provisioned with P40 and V100 GPUs. Let's create an `AmlCompute` cluster of `Standard_NC6s_v3` GPU VMs
* `max_nodes`: Maximum nodes to autoscale to while running a job on AmlCompute

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# choose a name for your cluster
gpu_cluster_name = 'gpu-cluster'

if gpu_cluster_name in ws.compute_targets:
    gpu_cluster = ws.compute_targets[gpu_cluster_name]
    if gpu_cluster and type(gpu_cluster) is AmlCompute:
        print('Found compute target. Will use {0} '.format(gpu_cluster_name))
else:
    print('creating new cluster')
    # m_size parameter below could be modified to one of the RAPIDS-supported VM types
    provisioning_config = AmlCompute.provisioning_configuration(vm_size = 'Standard_NC12s_v3',
                                                                max_nodes = 1,
                                                                idle_seconds_before_scaledown = 300,
                                                                vm_priority = "lowpriority")
    # Use VM types with more than one GPU for multi-GPU option, e.g. Standard_NC12s_v3
    
    # create the cluster
    gpu_cluster = ComputeTarget.create(ws, gpu_cluster_name, provisioning_config)
    
    # can poll for a minimum number of nodes and for a specific timeout 
    # if no min node count is provided it uses the scale settings for the cluster
    gpu_cluster.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)
    
# use get_status() to get a detailed status for the current cluster 
print(gpu_cluster.get_status().serialize())

### Use Custom Docker Image

We'll set up using a custom Docker Image using the `Environment` class. This can be used to install other packages necessary for the run. 

In [ ]:
from azureml.core import Environment

environment_name = "rapids"

env = Environment(environment_name)
env.docker.enabled = True
env.docker.base_image = None
env.docker.base_dockerfile = """
FROM rapidsai/rapidsai:0.19-cuda11.0-runtime-ubuntu18.04-py3.8
RUN apt-get update && \
apt-get install -y fuse && \
apt-get install -y build-essential && \
apt-get install -y python3-dev && \
source activate rapids && \
pip install azureml-defaults && \
pip install azureml-interpret && \
pip install interpret-community==0.18 && \
pip install azureml-telemetry
"""
env.python.user_managed_dependencies = True

### Create project directory

Create a directory that will contain all the necessary code from your local machine that you will need access to on the remote resource. This includes the training script, and any additional files your training script depends on.

The training script `train_explain.py` is already created for you. We'll move this to the project directory

In [ ]:
import os
import shutil

project_folder = './scripts'
os.makedirs(project_folder, exist_ok=True)
shutil.copy('train_explain.py', project_folder)

In [ ]:
from azureml.core import Run
from azureml.core import ScriptRunConfig

src = ScriptRunConfig(source_directory=project_folder, 
                      script='train_explain.py',
                      compute_target=gpu_cluster,
                      environment=env) 
run = experiment.submit(config=src)
run

Note: if you need to cancel a run, you can follow [these instructions](https://aka.ms/aml-docs-cancel-run).

In [ ]:
%%time
# Shows output of the run on stdout.
run.wait_for_completion(show_output=True)

In [ ]:
run.get_metrics()

## Download 
1. Download model explanation data.

In [ ]:
from azureml.interpret import ExplanationClient

# Get model explanation data
client = ExplanationClient.from_run(run)
global_explanation = client.download_model_explanation()
local_importance_values = global_explanation.local_importance_values
expected_values = global_explanation.expected_values


In [ ]:
# Or you can use the saved run.id to retrive the feature importance values
client = ExplanationClient.from_run_id(ws, experiment_name, run.id)
global_explanation = client.download_model_explanation()
local_importance_values = global_explanation.local_importance_values
expected_values = global_explanation.expected_values

In [ ]:
# Get the top k (e.g., 4) most important features with their importance values
global_explanation_topk = client.download_model_explanation(top_k=4)
global_importance_values = global_explanation_topk.get_ranked_global_values()
global_importance_names = global_explanation_topk.get_ranked_global_names()

In [ ]:
print('global importance values: {}'.format(global_importance_values))
print('global importance names: {}'.format(global_importance_names))

2. Download model file.

In [ ]:
# Retrieve model for visualization and deployment
from azureml.core.model import Model
import joblib
original_model = Model(ws, 'model_explain_model_on_amlcomp')
model_path = original_model.download(exist_ok=True)
original_model = joblib.load(model_path)

3. Download test dataset.

In [ ]:
# Retrieve x_test for visualization
import joblib
x_test_path = './x_test.pkl'
run.download_file('x_test.pkl', output_file_path=x_test_path)

In [ ]:
x_test = joblib.load('x_test.pkl')

## Visualize
Load the visualization dashboard

In [ ]:
from interpret_community.widget import ExplanationDashboard

In [ ]:
ExplanationDashboard(global_explanation, original_model, datasetX=x_test)

## Next
Learn about other use cases of the explain package on a:
1. [Training time: regression problem](https://github.com/interpretml/interpret-community/blob/master/notebooks/explain-regression-local.ipynb)       
1. [Training time: binary classification problem](https://github.com/interpretml/interpret-community/blob/master/notebooks/explain-binary-classification-local.ipynb)
1. [Training time: multiclass classification problem](https://github.com/interpretml/interpret-community/blob/master/notebooks/explain-multiclass-classification-local.ipynb)
1. Explain models with engineered features:
    1. [Simple feature transformations](https://github.com/interpretml/interpret-community/blob/master/notebooks/simple-feature-transformations-explain-local.ipynb)
    1. [Advanced feature transformations](https://github.com/interpretml/interpret-community/blob/master/notebooks/advanced-feature-transformations-explain-local.ipynb)
1. [Save model explanations via Azure Machine Learning Run History](../run-history/save-retrieve-explanations-run-history.ipynb)
1. Inferencing time: deploy a classification model and explainer:
    1. [Deploy a locally-trained model and explainer](../scoring-time/train-explain-model-locally-and-deploy.ipynb)
    1. [Deploy a locally-trained keras model and explainer](../scoring-time/train-explain-model-keras-locally-and-deploy.ipynb)
    1. [Deploy a remotely-trained model and explainer](../scoring-time/train-explain-model-on-amlcompute-and-deploy.ipynb)